# IRI Classification

based on https://towardsdatascience.com/transfer-learning-with-convolutional-neural-networks-in-pytorch-dd09190245ce

Imports

In [ ]:
!pip install efficientnet_pytorch

from IPython.core.interactiveshell import InteractiveShell
import seaborn as sns
# PyTorch
from torchvision import transforms, datasets, models
import torch
from torch import optim, cuda
from torch.utils.data import DataLoader, sampler
import torch.nn as nn
from efficientnet_pytorch import EfficientNet


import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# Data science tools
import numpy as np
import pandas as pd
import os
import csv
from shutil import copy,rmtree,copytree
from tqdm import tqdm
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from pprint import pprint
import random
#random = random.Random(17)

# Image manipulations
from PIL import Image

# Useful for examining network
from torchsummary import summary

# Timing utility
from timeit import default_timer as timer

# Visualizations
import matplotlib.pyplot as plt
#%matplotlib inline
plt.rcParams['font.size'] = 14

# Printing out all outputs
InteractiveShell.ast_node_interactivity = 'all'


Weights & Biases initialization

In [ ]:
# Weights and Biases is used to track and analyse results in their web-service. To disable it, comment out all lines containing "wandb"
!pip install --upgrade wandb --quiet
!wandb login (wandb credentials)
import wandb

### PARAMETERS

In [ ]:
#@title Chose ride/image combination
run_title = "Large Class" #@param {type:"string"}

ride_location = "sinsheim" #@param ["sinsheim", "karlsruhe", "ka_si"]
test_location = "same" #@param ["same", "sinsheim", "karlsruhe", "ka_si"]
image_construction = "C" #@param ["A","B_full","B_crop","B_top","C"]
test_image_construction = "C" #@param ["A","B_full","B_crop","B_top","C"]
MODEL_NAME = "efficientnet-b0" #@param ["vgg16", "resnet50", "efficientnet-b*", "resnet50_object_detection"] {allow-input: true}
number_of_repetitions =  1#@param {type:"integer"}

IRI_BINNING = True #@param {type:"boolean"}
OVERSAMPLING = "balanced_1.0" #@param ["True", "False", "\"balanced_1.0\""] {type:"raw", allow-input: true}
save_saliency = False #@param {type:"boolean"}

special_csv = "False" #@param ["False","fine","binary","mehrspurig außerorts","mehrspurig außerorts ohne erneuerte Fahrbahn", "extra features"]
drive_path = "/content/drive/My Drive/Masterthesis" #@param {type:"string"}

In [ ]:
# Training, validation, test-splits
splits = {
  "train_part" : 0.6,
  "val_part" : 0.2,
  "test_part" : 0.2
}
wandb_project = "rdd-iri-analysis" #"rdd-IRI-pytorch"

# Location of data
zip_path = '{}/Data/01 IRI prediction/99 zipped/{}_{}.zip'.format(drive_path,ride_location,image_construction).replace(' ', '\ ')


if image_construction != "A":  
  image_construction_csv = "BC"
else:
  image_construction_csv = "A"
csv_path = "{}/Data/01 IRI prediction/99 zipped/{}_{}_IRI.csv".format(drive_path,ride_location,image_construction_csv)

if special_csv == "mehrspurig außerorts":
  csv_path = drive_path + "/Data/01 IRI prediction/99 zipped/sinsheim_BC_IRI mehrspurig außerorts.csv"
  print(special_csv)
elif special_csv == "mehrspurig außerorts ohne erneuerte Fahrbahn":
  csv_path = drive_path + "/Data/01 IRI prediction/99 zipped/sinsheim_BC_IRI mehrspurig außerorts ohne erneuerte Fahrbahn.csv"
  print(special_csv)
elif special_csv == "fine":
  csv_path = drive_path + "/Data/01 IRI prediction/99 zipped/sinsheim_BC_IRI fine.csv"
  print(special_csv)
elif special_csv == "binary":
  csv_path = drive_path + "/Data/01 IRI prediction/99 zipped/sinsheim_BC_IRI binary.csv"
  print(special_csv)
elif special_csv == "extra features":
  csv_path = drive_path + "/Data/01 IRI prediction/99 zipped/sinsheim_BC_damages.csv"
  print(special_csv)


datadir = '/content/data/'
result_dir = drive_path + "/Results/Image analysis"
test_inferences_dir = drive_path + "/Results/Test inferences"
trained_resnet50_path = drive_path + "/Code/02 Object detection/trained_resnet50_backbone"

dirs = {
  "train": datadir + 'train/',
  "val": datadir + 'valid/',
  "test": datadir + 'test/'
}

# Model
BATCH_SIZE = 64 # Default: 128, best result: 64
FROZEN = True   # Freeze training of pretrained model weights
INPUT_SIZE = (600,600) 
DATA_BLOCKS = True # take sample blocks of 10 images for allocation to train-, validation- and test data
OVERSAMPLING_VAL_TEST = False # Oversampling of validation- and test-data

# Avoiding overfitting
N_EPOCHS = 120
MAX_EPOCHS_STOP = 30

save_file_name = MODEL_NAME + '_state-dict.pt'
checkpoint_path = MODEL_NAME + '_checkpoint.pt'

# Whether to train on a gpu
train_on_gpu = cuda.is_available()
print(f'Train on gpu: {train_on_gpu}')

multi_gpu = False
# Number of gpus
if train_on_gpu:
  gpu_count = cuda.device_count()
  print(f'{gpu_count} gpus detected.')
  if gpu_count > 1:
    multi_gpu = True
  else:
    multi_gpu = False

Train on gpu: True
1 gpus detected.


# Create training, validation & test dataset folders

In [ ]:
!mkdir -p $datadir
# copy images from zip folder in drive
!unzip -q $zip_path -d $datadir
os.chdir(datadir)
!find . -name '.DS_Store' -type f -delete
os.chdir("/content")

# Check if training and test dataset are the same
if test_location is not "same":
  print("Loading additional Test-Data for " + test_location)
  # Test dataset locations
  if test_image_construction is not "A":  
    test_image_construction_csv = "BC"
  else:
    test_image_construction_csv = "A"
  test_zip_path = '{}/Data/01 IRI prediction/99 zipped/{}_{}.zip'.format(drive_path,ride_location,test_image_construction).replace(' ', '\ ')
  test_csv_path = drive_path + "/Data/01 IRI prediction/{}_{}_IRI.csv".format(test_location,test_image_construction_csv)

  test_datadir = "/content/test_data/"
  test_dirs = {
    "train": test_datadir + 'train/',
    "val": test_datadir + 'valid/',
    "test": test_datadir + 'test/'
  }
  !mkdir -p $test_datadir
  # copy images from zip folder in drive
  !unzip -q $test_zip_path -d $test_datadir
  os.chdir(test_datadir)
  !find . -name '.DS_Store' -type f -delete
  os.chdir("/content")

In [ ]:
def read_csv(path):
  with open(path, mode='r') as infile:
    reader = csv.reader(infile)
    data_list = [rows for rows in reader]
  return data_list

def iri_binning(iri_list):
  # group IRI labels into three classes:
  # 1_3
  # 4
  # 5_6_7_8
  binned_iris = []
  for img,iri in iri_list:
    if iri in ['1','2','3'] :
      x = "iri_1_3"
    if iri in ['5','6','7','8']:
      x = "iri_5_8"
    if iri == '4':
      x = "iri_4"
    binned_iris.append([img,x])
  return binned_iris

def iri_binning_2(iri_list):
  # Group IRI labels into three classes, without upper limit for class iri_5__
  # 1_2_3
  # 4
  # 5_6_7_...
  binned_iris = []
  for img,iri in iri_list:
    iri = str(iri)
    if iri in ['1','2','3']:
      x = "iri_1_3"
    elif iri == '4':
      x = "iri_4"
    else:
      x = "iri_5__"  
    binned_iris.append([img,x])
  return binned_iris

# Randomly split and arrange data 
def split_up_data(classes,data,split_ranges):
  # Shuffle data and separate by classes
  data_by_class = []
  for c in classes:
    class_data = [d for d in data if d[1] == c]
    random.Random(17).shuffle(class_data)
    data_by_class.append(class_data)

  # Split data in training, validation and testing
  split_data = []
  
  for split in split_ranges:
    # iterate train,val,test
    split_d = []
    for d in data_by_class:
      # iterate classes
      range_from = round(split[0]*len(d))
      range_to = round(split[1]*len(d))
      split_d.append(d[range_from:range_to])
    split_data.append(split_d)
  return split_data

def split_up_data_blocks(classes,data,split_ranges,block_size = 10):
  # Create blocks of unshuffeled images
  data_blocks = []
  for i in range(0, len(data), block_size):
    data_blocks.append(data[i:i+block_size])

  # Randomly split data from blocks into training, validation and testing
  split_data = []
  len_data_blocks = len(data_blocks)
  random.Random(17).shuffle(data_blocks)
  # iterate train,val,test
  for split in split_ranges:
    split_d = []
    range_from = round(split[0]*len_data_blocks)
    range_to = round(split[1]*len_data_blocks)
    new_blocks = data_blocks[range_from:range_to]
    new_data = [d for b in new_blocks for d in b]
    split_d.extend(new_data)
    # Separate by classes
    data_by_class = []
    for c in classes:
      class_data = [d for d in split_d if d[1] == c]
      data_by_class.append(class_data)
    split_data.append(data_by_class)
  return split_data

### create class image folders for each iri value
def create_image_folders(csv_path, zip_path, IRI_BINNING, datadir, dirs_dict, splits, DATA_BLOCKS = True, OVERSAMPLING = False, OVERSAMPLING_VAL_TEST = False):
  print(csv_path)
  iri_csv = read_csv(csv_path)
  data = [[d[0],str(round(float(d[1])))] for d in iri_csv]
  print("Total images: {}".format(len(data)))

  if IRI_BINNING == 2:
    data = iri_binning_2(data)
  elif IRI_BINNING:
    data = iri_binning(data)
  
  
  splits_names = ["training", "validation", "test"]
  classes = set([d[1] for d in data])

  n_classes = len(classes)
  print("Classes: " + str(classes))
  
  # Make classnames with two digits
  if n_classes >= 10 and not IRI_BINNING:
    classes = [("0" + c)[-2:]  for c in classes]
    data = [[d[0],("0" + d[1])[-2:]] for d in data]
  
  classes = list(sorted(classes))
  
  # Define ranges of training, validation and test data
  split_ranges = [[0,splits["train_part"]],
            [splits["train_part"],splits["train_part"] + splits["val_part"]],
            [splits["train_part"] + splits["val_part"],1]]

  if DATA_BLOCKS:
    split_data = split_up_data_blocks(classes,data,split_ranges)
  else:
    split_data = split_up_data(classes,data,split_ranges)

  
  if OVERSAMPLING_VAL_TEST:
    oversampling_splits = [0,1,2]
  else:
    oversampling_splits = [0]
  
  for split_i in oversampling_splits:
    split_d = split_data[split_i] 

    if type(OVERSAMPLING) is not bool and OVERSAMPLING.startswith("balanced"):
      # Upsampling underrepresented & downsampling overrepresented classes
      sampling_parameter = float(OVERSAMPLING.split("_")[1])
      
      train_counts = [len(d) for d in split_d]
      average_class_size = sum(train_counts)/len(train_counts)
      for i,count in enumerate(train_counts):
        cl = classes[i]
        if count < average_class_size:
          # Replicate random training-images for underrepresented classes
          upsample = min(count,round(abs(average_class_size - count)*sampling_parameter))
          print("Oversampling "+str(upsample)+" "+ splits_names[split_i]+" images in class "+cl)
          random_sample = random.Random(42).sample([d for d in split_d[i] if d[1] == cl], upsample)
          split_d[i].extend(random_sample)
        else:
          # Remove random training-images from overrepresented classes
          downsample = round(abs(average_class_size - count)*sampling_parameter)
          print("Downsampling "+str(downsample)+" "+ splits_names[split_i]+" images in class "+cl)
          #print(downsample)
          for _ in range(downsample):
            split_d[i].remove(random.choice(split_d[i]))
          
    elif OVERSAMPLING:
      # Oversampling training images
      train_counts = [len(d) for d in split_d]
      max_train_count = max(train_counts)
      # Replicate random training-images for underrepresented classes
      for i,count in enumerate(train_counts):
        cl = classes[i]
        if count < max_train_count:
          upsample = max_train_count - count
          print("Oversampling "+str(upsample)+" "+ splits_names[split_i]+" images in class "+cl)
          random_sample = random.Random(42).sample([d for d in split_d[i] if d[1] == cl], upsample)
          split_d[i].extend(random_sample)

  # Create folders for train,val,test for each class

  size_dict = {}
  dirs = [d for _,d in dirs_dict.items()]
  old_datadir = os.path.join(datadir,zip_path.split("/")[-1].split(".")[0])

  for i,dir in enumerate(dirs):
    # iterate train,val,test paths
    sizes = {}
    try:
      rmtree(dir)
      os.mkdir(dir)
    except:
      os.mkdir(dir)

    for j,class_name in enumerate(tqdm(classes)):
      # iterate classes
      new_class_dir = os.path.join(dir, class_name)
      try:
        os.mkdir(new_class_dir)
      except:
        pass
      for img in split_data[i][j]:
        img_name = img[0]
        old_img_path = os.path.join(old_datadir,img_name)
        new_img_path = os.path.join(new_class_dir,img_name)
        if img_name in os.listdir(new_class_dir):
          new_img_path = new_img_path.split(".jpg")[0] + "_2.jpg"
        _ = copy(old_img_path,new_img_path)

      sizes[class_name] = len(os.listdir(new_class_dir))
    size_dict[dir] = sizes

  print(size_dict)

  return n_classes,size_dict



# Dataset creation

## Data Augmentation

Because there are a limited number of images in some categories, we can use image augmentation to artificially increase the number of images "seen" by the network. This means for training, we randomly resize and crop the images and also flip them horizontally. A different random transformation is applied each epoch (while training), so the network effectively sees many different versions of the same image. All of the data is also converted to Torch `Tensor`s before normalization. The validation and testing data is not augmented but is only resized and normalized. The normalization values are standardized for Imagenet. 

## Data Iterators

To avoid loading all of the data into memory at once, we use training `DataLoaders`. First, we create a dataset object from the image folders, and then we pass these to a `DataLoader`. At training time, the `DataLoader` will load the images from disk, apply the transformations, and yield a batch. To train and validation, we'll iterate through all the batches in the respective `DataLoader`. 

One crucial aspect is to `shuffle` the data before passing it to the network. This means that the ordering of the image categories changes on each pass through the data (one pass through the data is one training epoch).

### Batches

The shape of a batch is `(batch_size, color_channels, height, width)`. 

We can iterate through the `DataLoaders` when doing training, validation, and testing. This construction avoids the need to load all the data into memory and also will automatically apply the transformations to each batch. On each epoch, the `Random` transformations will be different so the network will essentially see multiple versions of each training image. 

In [ ]:
from torchvision.datasets import ImageFolder
class MyImageFolder(ImageFolder):
    """A generic data loader where the images are arranged in this way: ::

        root/dog/xxx.png
        root/dog/xxy.png
        root/dog/xxz.png

        root/cat/123.png
        root/cat/nsdf3.png
        root/cat/asd932_.png

    Args:
        root (string): Root directory path.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        loader (callable, optional): A function to load an image given its path.
        is_valid_file (callable, optional): A function that takes path of an Image file
            and check if the file is a valid file (used to check of corrupt files)

     Attributes:
        classes (list): List of the class names sorted alphabetically.
        class_to_idx (dict): Dict with items (class_name, class_index).
        imgs (list): List of (image path, class_index) tuples
    """

    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple: (sample, target, samplename) where target is class_index of the target class.
        """
        path, target = self.samples[index]
        sample = self.loader(path)
        sample_name = path.split("/")[-1]

        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)
            sample_name = self.target_transform(sample_name)

        return sample, target, sample_name

In [ ]:
def create_datasets(dirs, INPUT_SIZE, BATCH_SIZE):
  # Image transformations
  image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
      transforms.Resize(size=INPUT_SIZE),
      #transforms.RandomResizedCrop(size=INPUT_SIZE, scale=(0.8, 1.0), ratio = (1.0,1.3)),
      #transforms.RandomRotation(degrees=15),
      transforms.ColorJitter(),
      transforms.RandomHorizontalFlip(),
      #transforms.CenterCrop(size=TRANSFORM_SIZE),  # Image net standards
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406],
                          [0.229, 0.224, 0.225])  # Imagenet standards
    ]),
    # Validation does not use augmentation
    'val':
    transforms.Compose([
      transforms.Resize(size=INPUT_SIZE),
      #transforms.CenterCrop(size=TRANSFORM_SIZE),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    # Test does not use augmentation
    'test':
    transforms.Compose([
      transforms.Resize(size=INPUT_SIZE),
      #transforms.CenterCrop(size=TRANSFORM_SIZE),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
  }

  # Datasets from each folder
  data = {
    'train':
    MyImageFolder(root=dirs["train"], transform=image_transforms['train']),
    'val':
    MyImageFolder(root=dirs["val"], transform=image_transforms['val']),
    'test':
    MyImageFolder(root=dirs["test"], transform=image_transforms['test'])
  }

  # Dataloader iterators
  dataloaders = {
    'train': DataLoader(data['train'], batch_size=BATCH_SIZE, shuffle=True),
    'val': DataLoader(data['val'], batch_size=BATCH_SIZE, shuffle=True),
    'test': DataLoader(data['test'], batch_size=BATCH_SIZE, shuffle=False)
  }

  trainiter = iter(dataloaders['train'])
  features, labels,_ = next(trainiter)
  print(features.shape)
  print(labels.shape)

  print("Dataloaders created")
  return data, dataloaders

def imshow_tensor(image, ax=None, title=None):
  """Imshow for Tensor."""

  if ax is None:
      fig, ax = plt.subplots()

  # Set the color channel as the third dimension
  image = image.numpy().transpose((1, 2, 0))

  # Reverse the preprocessing steps
  mean = np.array([0.485, 0.456, 0.406])
  std = np.array([0.229, 0.224, 0.225])
  image = std * image + mean

  # Clip the image pixel values
  image = np.clip(image, 0, 1)

  ax.imshow(image)
  plt.axis('off')

  return ax, image

# Load Pre-Trained Model for Image Classification

- freeze the early layers of the pretrained model
- replace the classification module

## Approach

The approach for using a pre-trained image recognition model:

1. Load in pre-trained weights from a network trained on a large dataset
2. Freeze all the weights in the lower (convolutional) layers
    * Layers to freeze can be adjusted depending on similarity of task to large training dataset
3. Replace the classifier (fully connected) part of the network with a custom classifier
    * Number of outputs must be set equal to the number of classes
4. Train only the custom classifier (fully connected) layers for the task

In [ ]:
class CustomModelEfficientNet(nn.Module):
  # Class of EfficientNet B0 model with input of image and numeric data
  def __init__(self, n_classes, n_extra_features, dropout, frozen):
    # Load EfficientNet
    super(CustomModelEfficientNet, self).__init__()
    self.cnn = EfficientNet.from_pretrained("efficientnet-b0")

    # Set parameters to traineable
    for param in self.cnn.parameters():
        param.requires_grad = not frozen

    # Add on classifier in fully connected layer
    n_inputs = self.cnn._fc.in_features
    self.cnn._fc = nn.Linear(n_inputs, 256)
    
    # Add numeric extra-features
    self.fc1 = nn.Linear(256 + n_extra_features, 60)
    self.fc2 = nn.Linear(60, n_classes)

    self.dropout = dropout

  def forward(self, image, extra_features):
    x1 = self.cnn(image)
    x2 = extra_features
    # combine features from feature-extractor and numeric damage features
    x = torch.cat((x1, x2), dim=1)
    x = self.fc1(x)
    x = F.relu(x)
    x = F.dropout(x,p=self.dropout)
    x = self.fc2(x)
    x = F.softmax(x,dim=1)
    return x

def get_pretrained_model(model_name, multi_gpu, dropout, frozen = True):
  """Retrieve a pre-trained model from torchvision

  Params
  -------
      model_name (str): name of the model

  Return
  --------
      model (PyTorch model): cnn

  Available pretrained models in pytorch:
  --------
      resnet18 = models.resnet18(pretrained=True)
      alexnet = models.alexnet(pretrained=True)
      squeezenet = models.squeezenet1_0(pretrained=True)
      vgg16 = models.vgg16(pretrained=True)
      densenet = models.densenet161(pretrained=True)
      inception = models.inception_v3(pretrained=True)
      googlenet = models.googlenet(pretrained=True)
      shufflenet = models.shufflenet_v2_x1_0(pretrained=True)
      mobilenet = models.mobilenet_v2(pretrained=True)
      resnext50_32x4d = models.resnext50_32x4d(pretrained=True)
      wide_resnet50_2 = models.wide_resnet50_2(pretrained=True)
      mnasnet = models.mnasnet1_0(pretrained=True)
  """

  if model_name == 'vgg16':
    model = models.vgg16(pretrained=True)
    # Freeze early layers
    for param in model.parameters():
      param.requires_grad = not frozen
    # Add on classifier:
    #   Fully connected with ReLU activation (n_inputs, 256)
    #   Dropout with adjustable chance of dropping
    #   Fully connected with log softmax output (256, n_classes)
    classificator_size = 256
    n_inputs = model.classifier[6].in_features
    model.classifier[6] = nn.Sequential(
      nn.Linear(n_inputs, classificator_size), nn.ReLU(), nn.Dropout(dropout),
      nn.Linear(classificator_size, n_classes), nn.LogSoftmax(dim=1))
    
    # Printing classifier
    print("Classifier:")
    if multi_gpu:
      print(model.module.classifier)
    else:
      print(model.classifier)

  elif model_name == 'resnet50':
    model = models.resnet50(pretrained=True)
    for param in model.parameters():
      param.requires_grad = not frozen
    # Add on classifier in fully connected layer
    n_inputs = model.fc.in_features
    model.fc = nn.Sequential(
      nn.Linear(n_inputs, 256), nn.ReLU(), nn.Dropout(dropout),
      nn.Linear(256, n_classes), nn.LogSoftmax(dim=1))
  

  elif model_name == 'resnet50_object_detection':
    model = models.resnet50(pretrained=True)
    print(model)
    new_state_dict = torch.load(trained_resnet50_path)
    model.load_state_dict(new_state_dict)
    for param in model.parameters():
      param.requires_grad = not frozen
    # Add on classifier in fully connected layer
    n_inputs = model.fc.in_features
    model.fc = nn.Sequential(
      nn.Linear(n_inputs, 256), nn.ReLU(), nn.Dropout(dropout),
      nn.Linear(256, n_classes), nn.LogSoftmax(dim=1))
    
  elif model_name == 'mobilenet_v2':
    model = models.mobilenet_v2(pretrained=True)

    # Freeze early layers
    for param in model.parameters():
      param.requires_grad = not frozen
    # Add on classifier:
    n_inputs = model.classifier[1].in_features
    model.classifier = nn.Sequential(
      nn.Linear(n_inputs, 1280), nn.ReLU(), nn.Dropout(0.2),
      nn.Linear(1280, n_classes), nn.LogSoftmax(dim=1))
    # Printing classifier
    print("Classifier:")
    if multi_gpu:
      print(model.module.classifier)
    else:
      print(model.classifier)

  elif model_name.startswith('efficientnet-b'):
    if special_csv == "extra features":
      model = CustomModelEfficientNet(n_classes, n_extra_features, dropout, frozen)
    else:
      model = EfficientNet.from_pretrained(model_name)
      for param in model.parameters():
        param.requires_grad = not frozen
      # Add on classifier:
      n_inputs = model._fc.in_features
      model._fc = nn.Sequential(
        nn.Linear(n_inputs, 256), nn.ReLU(), nn.Dropout(dropout),
        nn.Linear(256, n_classes), nn.LogSoftmax(dim=1))

  # Move to gpu and parallelize
  if train_on_gpu:
    model = model.to('cuda')
  if multi_gpu:
    model = nn.DataParallel(model)

  # Mapping classes to indexes
  model.class_to_idx = data['train'].class_to_idx
  model.idx_to_class = {
    idx: class_
    for class_, idx in model.class_to_idx.items()
  }
  return model

# Saving Model

In the `train` function the `state_dict()`, the weights of the best model are saved. To save more information about the model, we use the below function. 

In [ ]:
def save_checkpoint(model, path):
    """Save a PyTorch model checkpoint

    Params
    --------
        model (PyTorch model): model to save
        path (str): location to save model. Must start with `model_name-` and end in '.pth'

    Returns
    --------
        None, save the `model` to `path`

    """

    model_name = path.split('-')[0]
    assert (model_name in ['vgg16', 'resnet50'
                           ]), "Path must have the correct model name"

    # Basic details
    checkpoint = {
        'class_to_idx': model.class_to_idx,
        'idx_to_class': model.idx_to_class,
        'epochs': model.epochs,
    }

    # Extract the final classifier and the state dictionary
    if model_name == 'vgg16':
        # Check to see if model was parallelized
        if multi_gpu:
            checkpoint['classifier'] = model.module.classifier
            checkpoint['state_dict'] = model.module.state_dict()
        else:
            checkpoint['classifier'] = model.classifier
            checkpoint['state_dict'] = model.state_dict()

    elif model_name == 'resnet50':
        if multi_gpu:
            checkpoint['fc'] = model.module.fc
            checkpoint['state_dict'] = model.module.state_dict()
        else:
            checkpoint['fc'] = model.fc
            checkpoint['state_dict'] = model.state_dict()

    # Add the optimizer
    checkpoint['optimizer'] = model.optimizer
    checkpoint['optimizer_state_dict'] = model.optimizer.state_dict()
        
    # Save the data to the path
    torch.save(checkpoint, path)

# Visualize saliency maps

Based on:
https://medium.com/datadriveninvestor/visualizing-neural-networks-using-saliency-maps-in-pytorch-289d8e244ab4 

In [ ]:
# Lists of used images
saliency_test_images = {
  "iri_1_3":[
              "20190819_14-39-18-092_sgm00139.jpg",
"20190819_13-39-58-707_sgm00080.jpg",
"20190819_14-39-18-092_sgm00057.jpg",
"20190819_12-50-08-839_sgm00403.jpg",
"20190819_12-50-08-839_sgm00402.jpg",
"20190819_10-32-50-611_sgm01176.jpg",
"20190816_11-08-09-342_sgm00098.jpg",
"20190816_10-30-47-864_sgm00029.jpg"
              ],
  "iri_4":[
            "20190814_14-44-24-433_sgm00060.jpg",
"20190819_10-32-50-611_sgm00878.jpg",
"20190819_12-00-05-082_sgm01279.jpg",
"20190819_10-32-50-611_sgm01008.jpg",
"20190819_10-32-50-611_sgm01020.jpg",
"20190819_13-57-11-762_sgm00191.jpg",
"20190819_14-39-18-092_sgm00202.jpg",
"20190819_14-39-18-092_sgm00052.jpg",
"20190819_13-39-58-707_sgm00088.jpg",
"20190819_11-15-57-991_sgm00095.jpg",
"20190819_11-15-57-991_sgm00184.jpg",
"20190816_11-08-09-342_sgm00009.jpg"
            ],
  "iri_5_8":[
              "20190814_14-44-24-433_sgm00063.jpg",
"20190819_12-50-08-839_sgm00589.jpg",
"20190814_14-44-24-433_sgm01330.jpg",
"20190819_10-32-50-611_sgm01154.jpg",
"20190819_13-39-58-707_sgm00101.jpg",
"20190819_11-15-57-991_sgm00123.jpg",
"20190819_11-15-57-991_sgm00180.jpg",
"20190819_13-39-58-707_sgm00016.jpg"]
              }


def save_saliency_maps(model,dirs,result_dir,transforms,saliency_test_images,run_name,epoch,f1_score):
  target_dir = os.path.join(result_dir,run_name,"saliency_map")
  try:
    os.makedirs(target_dir)
  except:
    pass

  # Preprocess image from test dataset
  trans = transforms.Compose([
    transforms.Resize(size=INPUT_SIZE),
    #transforms.CenterCrop(size=TRANSFORM_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
  ])
  
  # Run the model in evaluation mode
  _ = model.eval()

  for i,iri_class in enumerate(saliency_test_images):
    for image_name in saliency_test_images[iri_class]:
      # Open image
      image_path = os.path.join(dirs['test'],iri_class,image_name)    
      f1_round = round(f1_score,3)
      f1 = str(f1_round).replace(".",",")
      image_save_name = image_name.split(".jpg")[0] + "_original.jpg"
      
      image = Image.open(image_path)
      image.save(os.path.join(target_dir,image_save_name))

      image = trans(image).unsqueeze_(0).cuda()
      # Find the gradient with respect to the input image, so we need to call requires_grad_ on it
      _ = image.requires_grad_()

      # Forward pass through the model to get the prediction scores.
      prediction_scores = model(image)

      # Get the index corresponding to the maximum prediction score and the maximum prediction score itself.
      prediction_score_max_index = prediction_scores.argmax()
      prediction_score_max = prediction_scores[0,prediction_score_max_index]

      if prediction_score_max_index == i: 
        correct = 1
      else:
        correct = 0

      # Backward function on prediction_score_max performs the backward pass in the computation graph and calculates the gradient of 
      # prediction_score_max with respect to nodes in the computation graph
      prediction_score_max.backward()

      # Saliency is the gradient with respect to the input image now. But note that the input image has 3 channels,
      # R, G and B. To derive a single class saliency value for each pixel (i, j),  we take the maximum magnitude
      # across all colour channels.
      magnitudes = image.grad.data.abs()
      saliency, _ = torch.max(magnitudes,dim=1)
      # code to plot the saliency map as a heatmap
      fig = plt.figure()
      _ = plt.imshow(saliency[0].cpu(), cmap=plt.cm.hot)
      #plt.imshow(image.detach().cpu().squeeze(0)[0])
      _ = plt.axis('off')
      saliency_map_name = image_name.split(".jpg")[0] + "_epoch{}_act{}_pred{}_corr{}_acc{}.png".format(epoch,i,prediction_score_max_index,correct,f1)
      plt.savefig(os.path.join(target_dir,saliency_map_name),bbox_inches='tight')
      plt.close(fig)

# Training & Validation

### Training Loop:
- iterate through train `DataLoader`
- pass one batch through model
- After each batch: 
  - calculate the loss with `criterion(output, targets)`
  - calculate the gradients of the loss with respect to the model parameters with `loss.backward()`
  - call `optimizer.step()` to update the model parameters with the gradients
  - calculate dynamic learnrate with momentum via the optimizer *Adam*
- One complete pass through the training data: `epoch`
- after training loop:

### Validation loop:
- iterate through  validation `DataLoader`
- calculate classification quality parameter
- if quality increases: save model weights, to load a *best model* in the end
- if quality doesn't increase for a number of epochs: early stopping

In [ ]:
def weights_to_wandb(dictionary, filename):
  # Function to store sample weights to Weights and Biases
  df = pd.DataFrame.from_dict(dictionary, orient='index')
  df.reset_index(level=0, inplace=True)
  df.to_csv(filename, index=False, header=["Name","Weight"])
  wandb.save(filename)

def soft_accuracy(targets, outputs, margin = 1):
  # Calculate accuracy, but allow misclassifications within a certain margin
  correct = 0
  for target, output in zip(targets,outputs):
    if target - margin <= output <= target + margin:
      correct +=1
  acc = correct / len(targets)
  return acc


def train(model,
          learning_rate,
          train_loader,
          valid_loader,
          save_file_name,
          max_epochs_stop,
          loss_function='NLLLoss',
          weight_increase=None,
          n_epochs=50):
  """Train a PyTorch Model

  Params
  --------
      model (PyTorch model): cnn to train
      learning_rate (double): learning rate value of optimizer
      train_loader (PyTorch dataloader): training dataloader to iterate through
      valid_loader (PyTorch dataloader): validation dataloader used for early stopping
      save_file_name (str ending in '.pt'): file path to save the model state dict
      max_epochs_stop (int): maximum number of epochs with no improvement in validation loss for early stopping
      loss_function (nn Loss function)
      weight_increase (double): value >=0, that is added to the weight of missclassified samples
      n_epochs (int): maximum number of training epochs

  Returns
  --------
      model (PyTorch model): trained cnn with best weights
      history (DataFrame): history of train and validation loss and accuracy
  """
  # Define loss function
  if loss_function == "CrossEntropy":
    criterion = nn.CrossEntropyLoss(reduction='none')
  else:
    criterion = nn.NLLLoss(reduction='none')

  # Define Optimizer
  optimizer = optim.Adam(model.parameters(),lr=learning_rate)
  
  print("\nWeights being updated during training:")
  for p in optimizer.param_groups[0]['params']:
    if p.requires_grad:
      print(p.shape)

  # Early stopping intialization
  epochs_no_improve = 0
  valid_best_macro_fscore = 0
  
  valid_loss_min = np.Inf
  valid_best_acc = 0
  history = []

  labels = list(model.class_to_idx.keys())

  # Number of epochs already trained (if using loaded in model weights)
  try:
    print(f'Model has been trained for: {model.epochs} epochs.\n')
  except:
    model.epochs = 0
    print(f'Starting Training from Scratch.\n')


  overall_start = timer()

  # Initialize sample weights
  sample_weights = {s[0].split("/")[-1]:1 for s in dataloaders["train"].dataset.samples}

  # Main loop
  for epoch in range(n_epochs):

    # keep track of training and validation loss each epoch
    train_loss = 0.0
    valid_loss = 0.0

    train_acc = 0
    valid_acc = 0

    # Set to training
    model.train()
    start = timer()

    # Training loop
    for ii, (data, target, sample_names) in enumerate(train_loader):
      # Tensors to gpu
      if train_on_gpu:
        data, target = data.cuda(), target.cuda()

      # Clear gradients
      optimizer.zero_grad()
      # Predicted outputs are log probabilities
      output = model(data)

      # Loss calculation
      loss = criterion(output, target)
      
      # Increase weight of missclassified samples
      if weight_increase:
        _, predicted = torch.max(output.data, 1)
        check_list = zip(predicted,target,sample_names)
        sample_weights_batch = []
        for pred,targ,s_name in check_list:
          # Check if samples in batch were misclassified
          if pred != targ:
            sample_weights[s_name] += weight_increase
          # Create list of sample weights in current batch
          sample_weights_batch.append(sample_weights[s_name])
        # Turn weight vector into tensor
        sample_weights_batch = torch.FloatTensor(sample_weights_batch).to('cuda')
        # Apply weights onto loss
        loss = (loss * sample_weights_batch / sample_weights_batch.sum()).sum()
        # Backpropagation of gradients
        loss.backward()
      else:
        # Continue without sample weights
        loss = loss.mean()
        # Backpropagation of gradients
        loss.backward()

      # Update the parameters
      optimizer.step()

      # Track train loss by multiplying average loss by number of examples in batch
      train_loss += loss.item() * data.size(0)

      # Calculate accuracy by finding max log probability
      _, pred = torch.max(output, dim=1)
      correct_tensor = pred.eq(target.data.view_as(pred))
      # Need to convert correct tensor from int to float to average
      accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
      # Multiply average accuracy times the number of examples in batch
      train_acc += accuracy.item() * data.size(0)
    
      # Track training progress
      print(
          f'Epoch: {epoch}\t{100 * (ii + 1) / len(train_loader):.2f}% complete. {timer() - start:.2f} seconds elapsed in epoch.',
          end='\r')      

    # After training loop ends, start validation
    else:
      model.epochs += 1
      
      outputs = []
      output_probs = []
      targets = []

      # Don't need to keep track of gradients
      with torch.no_grad():
        # Set to evaluation mode
        model.eval()

        # Validation loop
        for data, target, sample_names in valid_loader:
          # Tensors to gpu
          if train_on_gpu:
            data, target = data.cuda(), target.cuda()

          # Forward pass
          output = model(data)
          
          # Track targets and predictions
          out = np.argmax(np.array(output.tolist()),axis=1)
          outputs.extend(out)
          output_probs.extend(output.tolist())
          targets.extend(target.tolist())

          # Validation loss
          loss = criterion(output, target)
          # Multiply average loss times the number of examples in batch
          valid_loss += loss.mean().item() * data.size(0)

          # Calculate validation accuracy
          _, pred = torch.max(output, dim=1)
          correct_tensor = pred.eq(target.data.view_as(pred))
          accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
          
          # Multiply average accuracy times the number of examples
          valid_acc += accuracy.item() * data.size(0)                    


      # Calculate average losses
      train_loss = train_loss / len(train_loader.dataset)
      valid_loss = valid_loss / len(valid_loader.dataset)

      # Calculate average accuracy
      train_acc = train_acc / len(train_loader.dataset)
      valid_acc = valid_acc / len(valid_loader.dataset)

      history.append([train_loss, valid_loss, train_acc, valid_acc])

      # Calculate soft accuracy
      soft_valid_acc = soft_accuracy(targets, outputs)

      # Calculate validation quality parameters
      macro_precision, macro_recall, macro_fscore,_ = precision_recall_fscore_support(targets, outputs, average='macro')
      micro_precision, micro_recall, micro_fscore,_ = precision_recall_fscore_support(targets, outputs, average='micro')
      
      # Log to weights & biases
      wandb.sklearn.plot_confusion_matrix(targets, outputs, labels)
      wandb.log({'pr': wandb.plots.precision_recall(targets, output_probs, labels)})
      wandb.log({'Training loss': train_loss,
                  "Training accuracy": train_acc,
                  "Validation loss": valid_loss,
                  "Validation accuracy": valid_acc,
                  "Validation soft accuracy": soft_valid_acc,
                  "Precision macro": macro_precision,
                  "Precision micro": micro_precision,
                  "Recall macro": macro_recall,
                  "Recall micro": micro_recall,
                  "F-Score macro": macro_fscore,
                  "F-Score micro": micro_fscore,
                  'Epoch': epoch})                

      # Calculate saliency maps
      if save_saliency:
          save_saliency_maps(model,dirs,result_dir,transforms,saliency_test_images,wandb.run.name,epoch,macro_fscore)

      # Print training and validation results
      if (epoch + 1) % 100 == 0:
        print(
            f'\nEpoch: {epoch} \tTraining Loss: {train_loss:.4f} \tValidation Loss: {valid_loss:.4f}'
        )
        print(
            f'\t\tTraining Accuracy: {100 * train_acc:.2f}%\t Validation Accuracy: {100 * valid_acc:.2f}%'
        )

      # Track best validation accuracy     
      if valid_acc > valid_best_acc:
        valid_best_acc = valid_acc
        wandb.run.summary["best_accuracy"] = valid_best_acc

      # Save the model if  macro F-Score increases
      if macro_fscore > valid_best_macro_fscore:
        valid_best_macro_fscore = macro_fscore
        wandb.run.summary["Best F-Score macro"] = macro_fscore
        # Save model (weights)
        torch.save(model.state_dict(), save_file_name)
        wandb.save(save_file_name)
        print("state_dict saved 👌🏻")
        # Track improvement
        epochs_no_improve = 0          
        best_epoch = epoch
        wandb.run.summary["best_epoch"] = best_epoch

      # Otherwise increment count of epochs with no improvement
      else:
        epochs_no_improve += 1
        # Trigger early stopping
        if epochs_no_improve >= max_epochs_stop:
          print(
              f'\nEarly Stopping! Total epochs: {epoch}. Best epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_best_acc:.2f}%'
          )
          total_time = timer() - overall_start
          print(
              f'{total_time:.2f} total seconds elapsed. {total_time / (epoch+1):.2f} seconds per epoch.'
          )

          # Load the best state dict
          model.load_state_dict(torch.load(save_file_name))
          # Attach the optimizer
          model.optimizer = optimizer

          # Format history
          history = pd.DataFrame(
              history,
              columns=[
                  'train_loss', 'valid_loss', 'train_acc',
                  'valid_acc'
              ])
          
          # Save sample weights to wandb
          weights_to_wandb(sample_weights,"sample_weights.csv")

          return model, history, sample_weights

  # Attach the optimizer
  model.optimizer = optimizer
  # Record overall time and print out stats
  total_time = timer() - overall_start
  print(
      f'\nBest epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%'
  )
  print(
      f'{total_time:.2f} total seconds elapsed. {total_time / (epoch):.2f} seconds per epoch.'
  )

  # Save checkpoint (overall model info)
  try:
    save_checkpoint(model,checkpoint_path)
    wandb.save(checkpoint_path)
  except:
    pass
  wandb.save(save_file_name)

  # Save sample weights to wandb
  weights_to_wandb(sample_weights,"sample_weights.csv")

  # Format history
  history = pd.DataFrame(
      history,
      columns=['train_loss', 'valid_loss', 'train_acc', 'valid_acc'])
  return model, history, sample_weights

Testing

In [ ]:
def test_inference(model,dataloaders):
  # extract specific run-name from weights & biases
  run_name = wandb.run.name.split("-")
  run_name = run_name[2] + "-" + run_name[0] + "-" + run_name[1]
  # create filepaths to store .csv results
  target_path_1 = os.path.join(test_inferences_dir, run_name + '.csv')
  target_path_2 = os.path.join(test_inferences_dir, run_name + '_confidences.csv')

  # Load best model
  model.load_state_dict(torch.load(save_file_name))

  # Initialize list of results
  test_inference_results = []
  header_1 = ["Filename","Target","Prediction","Confidence","Confidence Vector"]

  # Set model to inference-only (no training)
  with torch.no_grad():
    model.eval()
    # Iterate testdata
    for i, (images, labels, sample_names) in tqdm(enumerate(dataloaders["test"], 0)):
      # Get inference results
      outputs = model(images.to('cuda'))
      _, predicted = torch.max(outputs.data, 1)
      # Get classification probabilities
      probabilities = outputs.data.tolist()
      batch_size = len(predicted)
      for j,pred_item in enumerate(predicted.tolist()):
        # Collect target info
        sample_name, sample_target = dataloaders["test"].dataset.samples[i*batch_size + j]
        sample_name = sample_name.split("/")[-1]
        # Calculate confidence
        confidence = np.exp(max(probabilities[j]))
        # Consolidate result info
        line = [sample_name,sample_target,pred_item,confidence]
        test_inference_results.append(line)

  # Save result info in DataFrame
  inference_df = pd.DataFrame(test_inference_results)
  # Save result info as .csv
  inference_df.to_csv(target_path_1, index=False, header=header_1)
  # Save .csv to weights & biases
  inference_df.to_csv("test_inference_results.csv", index=False, header=header_1)
  wandb.save('test_inference_results.csv')
  
  # Track classification quality to weights & biases
  targets = [i[1] for i in test_inference_results]
  outputs = [i[2] for i in test_inference_results]
  accuracy = accuracy_score(targets, outputs)
  # Calculate test classification quality parameters
  macro_precision, macro_recall, macro_fscore,_ = precision_recall_fscore_support(targets, outputs, average='macro')
  wandb.run.summary["Test Accuracy"] = accuracy
  wandb.run.summary["Test Macro F-Score"] = macro_fscore
  wandb.run.summary["Test Macro Precision"] = macro_precision
  wandb.run.summary["Test Macro Recall"] = macro_recall
  print("Test Accuracy: " + str(accuracy))

  # Calculate confidence chart
  conf_graph_data = []
  header_2 = ["Conficence","Accuracy","Macro Precision","Macro Recall","Macro F-Score","Percentage predicted"]
  # Iterate confidence threshold steps
  for c in range(1,100,2):
    subset = [[targ,pred] for _,targ,pred,conf in test_inference_results if conf >= c/100]
    if subset:
      targets, outputs = zip(*subset)
      accuracy = accuracy_score(targets, outputs)
      macro_precision, macro_recall, macro_fscore,_ = precision_recall_fscore_support(targets, outputs, average='micro')
      perc_predicted = len(subset)/len(test_inference_results)
      conf_graph_data.append([c/100,round(accuracy,2),round(macro_precision,2),round(macro_recall,2),round(macro_fscore,2),round(perc_predicted,2)])
  # save confidence chart
  conf_graph_df = pd.DataFrame(conf_graph_data)
  conf_graph_df.to_csv(target_path_2, index=False, header=header_2)
  conf_graph_df.to_csv("test_inference_results_confidences.csv", index=False, header=header_2)
  wandb.save('test_inference_results_confidences.csv')

  


# MAIN
This cell runs the functions that were defined above

In [ ]:
# Create image folders of the dataset
n_classes,size_table = create_image_folders(csv_path, zip_path, IRI_BINNING, datadir, dirs, splits, DATA_BLOCKS, OVERSAMPLING, OVERSAMPLING_VAL_TEST)

# If foreign test data is used, replace the test image folders
if test_location is not "same":
  print("Creating special Test-Set for " + test_location)
  test_location_splits = {"train_part" : 0.6,  "val_part" : 0.2,  "test_part" : 0.2}
  create_image_folders(test_csv_path, test_zip_path, IRI_BINNING, test_datadir, test_dirs, test_location_splits, DATA_BLOCKS, OVERSAMPLING, OVERSAMPLING_VAL_TEST)
  #overwrite original test data with selected test data
  rmtree(dirs["test"])
  copytree(test_dirs["test"],dirs["test"])

# Create datasets and dataloaders
data, dataloaders = create_datasets(dirs, INPUT_SIZE, BATCH_SIZE)

#### Start training

In [ ]:
training_combinations = [
                         #[0.0001,0.4,0],
                         [0.0001,0.4,0.0001],#######
                         #[0.0001,0.5,0],
                         #[0.0001,0.5,0.0001],
                         #[0.001,0.4,0],
                         #[0.001,0.4,0.0001],
                         #[0.001,0.5,0],
                         #[0.001,0.5,0.0001],
                         #[0.01,0.4,0],
                         #[0.01,0.4,0.0001],
                         #[0.01,0.5,0],
                         #[0.01,0.5,0.0001]
                         ]

# Iterate through training parameters and run training procedure
run_count = 0
for LR,DROPOUT,WEIGHT_INCREASE in training_combinations:
  for repetition in range(number_of_repetitions):
    run_count += 1
    print("+++++  Starting run {}. (LR:{}, Dropout:{}, Sample Weights:{})".format(run_count,LR,DROPOUT,WEIGHT_INCREASE))
    
    # Load pretrained model
    model = get_pretrained_model(MODEL_NAME, multi_gpu, DROPOUT, FROZEN)
    
    # Initialize Weights & Biases run
    wandb.init(project=wandb_project, reinit=True, entity="rdd")
    try:
      wandb.watch(model)
    except:
      pass

    training_parameters = (
        model,
        LR,
        dataloaders['train'],
        dataloaders['val'],
        save_file_name,
        MAX_EPOCHS_STOP,
        'NLLLoss',
        WEIGHT_INCREASE,
        N_EPOCHS)

    # Document parameter configuration to Weights & Biases
    wandb.config.update({
        "Model name": MODEL_NAME,
        "Training batch size": BATCH_SIZE,
        "Learning rate": LR,
        "Max number of epochs": N_EPOCHS,
        "Max epochs stop": MAX_EPOCHS_STOP,
        "Input image size": INPUT_SIZE,
        "Frozen pretrained weights": FROZEN,
        "Dropout percentage": DROPOUT,
        "Sample weight increase": WEIGHT_INCREASE,
        "IRI Binning": IRI_BINNING,
        "Blockwise data sampling": DATA_BLOCKS,
        "Oversampling": OVERSAMPLING,
        "Oversampling Val & Test":OVERSAMPLING_VAL_TEST,
        "Training splits": splits,
        "IRI csv file": csv_path,
        "Image dataset": zip_path,
        "Train location": ride_location,
        "Test location": test_location,
        "Run title": run_title
        })
    
    # Start training
    model, history, sample_weights = train(*training_parameters)

    # Get test results
    test_inference(model,dataloaders)
    
    try:
      # Track model info on Weights & Biases
      wandb.watch(model)
    except:
      pass